In [1]:
import numpy as np
from boto.s3.connection import S3Connection
from boto.s3.key import Key
import boto
import sys, os
import pandas as pd
from textblob import Blobber
from textblob_aptagger import PerceptronTagger
import re
import editdistance
import datetime
import subprocess

In [289]:
today = datetime.date.today()
yr = str(today.year)[-2:]
mnth = today.month
if len(str(mnth)) == 1:
    mnth = '0' + str(mnth)
else:
    mnth = str(mnth)
day = today.day
if len(str(day)) == 1:
    day = '0' + str(day)
else:
    day = str(day)

url = "s3://mids-capstone-rzst/es_staging/" + str(yr) + '.' + str(mnth) + '.' + str(day) + '/es_data.csv'


17.03.29


In [2]:
full_dataset = pd.read_csv('~/w210_imwithdata/data/bucket_stuff/es_data.csv')
print full_dataset.columns
tb = Blobber(pos_tagger=PerceptronTagger())
print len(full_dataset)

Index([u'issue', u'action', u'id', u'es_score', u'tweet_timestamp',
       u'query_timestamp', u'tweet_user', u'tweet_cities', u'tweet_states',
       u'tweet_urls', u'tweet_phone_numbers', u'tweet_dates_ref',
       u'tweet_legislator_names', u'tweet_legislator_handles', u'tweet'],
      dtype='object')
21256


In [5]:
test_data = pd.read_csv('~/w210_imwithdata/data/static_data/full_rzst_test_dataset.csv')
print len(test_data)
print test_data.columns


16634
Index([u'issue', u'action', u'id', u'es_score', u'tweet_timestamp',
       u'query_timestamp', u'tweet_user', u'tweet', u'actionable'],
      dtype='object')


In [25]:
actionability_ranking = full_dataset[['issue','action','tweet','es_score','tweet_timestamp','tweet_cities','tweet_states','tweet_urls'
                                      ,'tweet_phone_numbers','tweet_dates_ref','tweet_legislator_names','tweet_legislator_handles']]

actionability_ranking[:10]

# print actionability_ranking.action.unique()

,issue,action,tweet,es_score,tweet_timestamp,tweet_cities,tweet_states,tweet_urls,tweet_phone_numbers,tweet_dates_ref,tweet_legislator_names,tweet_legislator_handles
0,civil_rights,townhall,@ACLU Uhh... Town hall rules vs. freedom of sp...,15.114339,2017-03-28T23:58:46.000Z,[],[],['https://t.co/LO1D2VoXkZ'],[],[],[],[]
1,civil_rights,townhall,Rep. John Culberson (R-TX) did not expect a to...,13.396873,2017-03-28T15:12:36.000Z,[],[],['https://t.co/9hyDDc48V5'],[],[],[],[]
2,civil_rights,townhall,@IndivisibleNC @ACLU @reppittenger not surpris...,10.583446,2017-03-28T16:07:16.000Z,[],[],[],[],[],[],[]
3,civil_rights,townhall,We need ACLU supporters to attend the #Florida...,10.583446,2017-03-28T18:58:45.000Z,[],[],['https://t.co/purKwkA1PR'],[],[],[],[]
4,civil_rights,townhall,We need ACLU supporters to attend the #Florida...,10.156322,2017-03-28T18:59:17.000Z,[],[],['https://t.co/xLm9uqZsfd'],[],[],[],[]
5,civil_rights,townhall,We need ACLU supporters to join us at the Cons...,9.867394,2017-03-28T14:34:25.000Z,[],[],['https://t.co/iwWrsLSOAn'],[],[],[],[]
6,civil_rights,townhall,We need ACLU supporters to join us at the Cons...,9.733502,2017-03-28T18:31:01.000Z,[],[],['https://t.co/c1Jp7v7wjY'],[],[],[],[]
7,civil_rights,townhall,Need swag for the #MarchforScience? How about ...,9.364697,2017-03-28T16:01:45.000Z,[],[],['https://t.co/wWD8QAcf5h'],[],[],[],[]
8,civil_rights,townhall,@MissGingerDawn @ACLU @RepAndyHarrisMD Stephen...,6.460799,2017-03-28T13:48:35.000Z,[],[],[],[],[],[],[]
9,speech,townhall,.@RepChrisStewart town hall Friday 6:30 @Wes...,19.615380,2017-03-28T00:27:52.000Z,[],[],['https://t.co/sjyhVBVmt7'],[],[],[],[]


In [139]:
profanity = ['anal', 'anus','ass','asshole','asshole.' 'ballsack', 'blowjob', 'blow job', 'boner', 'clitoris', 'cock', 'cunt', 'dick', 'dildo', 'dyke', 'fag', 'fuck', 'jizz', 'labia', 'muff', 'nigger', 'nigga','nigg', 'penis', 'piss', 'pussy', 'scrotum', 'sex', 'shit', 'slut', 'smegma', 'spunk', 'twat', 'vagina', 'wank', 'whore']
profanity_regex = re.compile(r'(' + '|'.join(profanity) + r')', re.IGNORECASE)

In [234]:
language_scores = []
bad_verbs = ['VBZ','VBN','VBD','RB']
good_verbs = ['VB','VBG','VBP','JJ']


### Part of speech tag each tweet
for tweet in actionability_ranking['tweet'].tolist():
    tagged = tb(tweet.decode('latin-1').lower())
    tag_list = [x[1] for x in tagged.tags]
#     print tweet, tag_list
    score = 0
    
    ### Penalize tweets with structures that are known to be now what we're looking for
    ### POS Tags available here: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
    if tag_list[:2] == ['NNP','POS',]:
        score -= 4
    if tag_list[:3] == ['NN', 'POS','NN']:
        score -= 10
    if tag_list[:3] == ['NN', 'NN', 'TO']:
        score -= 10
    if tag_list[:3] == ['NN', 'TO', 'VB']:
        score -= 10
    if tag_list[:3] == ['NN', 'NN', 'VBZ']:
        score -= 10
    if tag_list[:3] == ['NN', 'NN', 'JJ']:
        score -= 10
    if tag_list[:3] == ['NN', 'JJ', 'NN']:
        score -= 10
    
    if tag_list[:3] == ['JJ', 'NNS', 'VBP']:
        score -= 4
    elif tag_list[:3] == ['JJ', 'NN', 'VBG']:
        score -= 4
    elif tag_list[:3] == ['JJS', 'NN', 'JJ']:
        score -= 4
    elif tag_list[:3] == ['NN', 'NN', 'NN']:
        score -= 4
    elif tag_list[:3] == ['JJ', 'NN', 'NN']:
        score -= 4
    elif tag_list[:3] == ['JJ', 'NN', 'NNS']:
        score -= 4
    elif tag_list[:3] == ['JJ', 'NN', 'VB']:
        score -= 10
    elif tag_list[:3] == ['IN', 'JJ', 'NN']:
        score -= 10
    elif tag_list[0] == 'JJ':
        score += 3
        
    if tag_list[0] == 'VB':
        score += 4
    
    
    #### Penalize tweets with words in them that indicate they are not useful
    if 'yesterday' in tweet.lower():
        score -= 10
    if 'last week' in tweet.lower():
        score -= 10
    if 'video' in tweet.lower():
        score -= 10
    if 'news search' in tweet.lower():
        score -= 10
    if 'town hall in facebook' in tweet.lower():
        score -= 20
    if 'townhall in facebook' in tweet.lower():
        score -= 20
    if "facebook's" in tweet.lower() and "town hall" in tweet.lower():
        score -= 20
    if 'cnn town hall' in tweet.lower():
        score -= 20
    if 'online  paper' in tweet.lower():
        score -= 20
    if 'join us' in tweet.lower():
        score -= 10
    if 'sales' in tweet.lower():
        score -= 20
    if 'relaxing' in tweet.lower():
        score -= 20
    if 'canada' in tweet.lower():
        score -= 20
    if 'minister' in tweet.lower():
        score -= 20
    if 'UK' in tweet:
        score -= 20
    if 'MP' in tweet:
        score -= 20
    if 'EU' in tweet:
        score -= 20
    if 'england' in tweet.lower():
        score -= 20 
    if 'germany' in tweet.lower():
        score -= 20 
    if 'south africa' in tweet.lower():
        score -= 20 
    if 'News' in tweet:
        score -= 10
    if 'Daily Beast' in tweet.lower():
        score -= 20
    if '#design' in tweet.lower():
        score -= 20
    if '#interior' in tweet.lower():
        score -= 20
    if 'radicalisation' in tweet.lower():
        score -= 20
    if 'militancy' in tweet.lower():
        score -= 20
    if 'sharia' in tweet.lower():
        score -= 20
    if 'uncontrolled' in tweet.lower():
        score -= 20
    if 'enjoy a' in tweet.lower():
        score -= 20
    if tweet.lower().startswith('the latest'):
        score -= 20
    if 'wapo' in tweet.lower():
        score -= 20
    if 'nytimes' in tweet.lower():
        score -= 20
    if '5 things' in tweet.lower():
        score -= 20
    if 'poland' in tweet.lower():
        score -= 20
    if 'hungary' in tweet.lower():
        score -= 20
    if 'slovakia' in tweet.lower():
        score -= 20
    if 'czech' in tweet.lower():
        score -= 20
    if 'hiring' in tweet.lower():
        score -= 20
    if 'ice show' in tweet.lower():
        score -= 20
    if 'ice cream' in tweet.lower():
        score -= 20
    if 'ice cold' in tweet.lower():
        score -= 20
    if 'snow' in tweet.lower():
        score -= 20
    if 'alex jones' in tweet.lower():
        score -= 1000
    if re.findall(profanity_regex,tweet.lower()):
        score -= 50
    
    ### Penalize tweets with tons of hashtags
    if tweet.count('#') > 2:
        score -= (tweet.count('#') - 2) * 2
    ### Penalize tweets with tons of mentions
    if tweet.count('@') > 2:
        score -= (tweet.count('@') - 2) * 2
    
    ### Reward tweets with good verbs and no bad verbs
    verb_score = 0
    for tag in tag_list:
        if tag in bad_verbs:
            verb_score -= 2
        if tag in good_verbs:
            verb_score += 1
    score += int(verb_score * 1.0 / len(tag_list)) * 2
    
    ### Reward polite tweets
    if tweet.lower().startswith('please'):
        score += 3
    
    ### Other score adjustments
    # Reward tweets that are longer
    score += int(len(tweet)/60)
    
    language_scores.append(score)
        

    


In [235]:
print len(actionability_ranking['tweet'])
print len(language_scores)

actionability_ranking['pos_score'] = np.asarray(language_scores)


21256
21256


In [236]:
actionability_ranking['actionability_score'] = (np.where(actionability_ranking['tweet_cities'] == '[]', 0, 4) + 
                        np.where(actionability_ranking['tweet_states'] == '[]', 0, 2) + 
                       np.where(actionability_ranking['tweet_urls'] == '[]', 0, 1) + 
                       np.where(actionability_ranking['tweet_phone_numbers'] == '[]', 0, 4) + 
                       np.where(actionability_ranking['tweet_dates_ref'] == '[]', 0, 8) + 
                       np.where(actionability_ranking['tweet_legislator_names'] == '[]', 0, 1) + 
                       np.where(actionability_ranking['tweet_legislator_handles'] == '[]', 0, 1) + 
                       np.where(actionability_ranking['tweet'].str.startswith('@'), -10, 0) + 
                        np.where(actionability_ranking['tweet'].str.startswith('.@'), -10, 0) 
                       ) 


actionability_ranking['total_score'] = actionability_ranking['es_score'] + actionability_ranking['actionability_score']  + actionability_ranking['pos_score']
actionability_ranking.sort(['total_score'], ascending=[0], inplace=True)

/home/brosskatz/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:14: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [237]:
print sum(pos_scores) / float(len(pos_scores))
print actionability_ranking['actionability_score'].median()
print actionability_ranking['es_score'].median()

2.41821400634
1.0
4.7697415


In [238]:
actionability_ranking['tweet'].tolist()[:5000]

['Take Action: Sign Petition to Confirm you Still Want a Secure Border which includes Building a Wall Immediately https://t.co/tm3KqlGb2S',
 'Take Action: Sign Petition to Confirm you Still Want a Secure Border which includes Building a Wall Immediately https://t.co/qTtqqIuT7l!',
 'Sign Petition to Have City Officials of Sanctuary Cities Held Legally and Financially Responsible  for Crimes  https://t.co/d7reyZTQeI',
 'Sign Petition 2 Have City Offs. of Sanctuary Cities Held Legally &amp; Fin Responsible 4 Crimes Committed by Illegals https://t.co/bWW1aA8jN1',
 'Sign the petition: Stop Republican legislation to eliminate the EPA https://t.co/UR43JDfVWI via @CREDOMobile #p2 please sign petition!',
 'Sign Petition to Have City Officials of Sanctuary Cities Held Legally and Financially Responsible for Crimes... https://t.co/MGixnQWYpN',
 'Take Action: Sign Petition to Have City Officials of Sanctuary Cities Held Legally and Financially Responsible  for Crimes by Illegals',
 'Take Our Citie

In [248]:
# es_score_list = actionability_ranking['es_score'].tolist()
# issue_list = actionability_ranking['issue'].tolist()
# action_list = actionability_ranking['action'].tolist()
score_list = actionability_ranking['total_score'].tolist()
# action_score_list = actionability_ranking['actionability_score'].tolist()
# language_score_list = actionability_ranking['pos_score'].tolist()
# tweet_list = actionability_ranking['tweet'].tolist()

In [244]:
# for i,tweet in enumerate(actionability_ranking['tweet'].tolist()):
#     if score_list[i] > 7.5 and  es_score_list[i] > 5.5:
#         tagged = tb(tweet.decode('latin-1').lower())
#         tag_list = [x[1] for x in tagged.tags]
#         print tweet, es_score_list[i], score_list[i], tag_list
#         print
#         if i % 100 == 0:
#             print i
#             print 

In [291]:
filtered_data =  actionability_ranking.loc[(actionability_ranking['total_score'] > 8.5) | (actionability_ranking['es_score'] > 7.0)]
filtered_tweet_list = filtered_data['tweet'].tolist()
print len(filtered_tweet_list)
filtered_score_list = filtered_data['total_score'].tolist()
print len(filtered_score_list)
filtered_es_score_list = filtered_data['es_score'].tolist()

4263
4263


In [296]:
distance_dict = {} 

for i, tweet in enumerate(filtered_tweet_list):
    for j, tweet2 in enumerate(filtered_tweet_list):
        if score_list[i] > 7.5 and  es_score_list[i] > 5.5:
            tweet_ids = tuple(sorted([i,j]))
    #             distance_list.append(int(editdistance.eval(tweet,tweet2)))
            if i == j:
                pass
            else:
                distance = editdistance.eval(tweet,tweet2)
                if distance <= 60:
                    distance_dict[tweet_ids] = distance
            
            

In [297]:
len(distance_dict)

159824

In [298]:
delete_indices = []

for (i,j), v in distance_dict.iteritems():
    if filtered_score_list[i] >= filtered_score_list[j]:
        delete_indices.append(j)
    else:
        delete_indices.append(i)

delete_indices = list(set(delete_indices))
print len(delete_indices)

2179


In [299]:
filtered_data.drop(filtered_data.index[delete_indices], inplace = True, errors='ignore')
print len(filtered_data)

2084


/home/brosskatz/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [300]:
# final_data = filtered_data[[u'issue', u'action', u'id', u'es_score', u'tweet_timestamp',
#        u'query_timestamp', u'tweet_user', u'tweet_cities', u'tweet_states',
#        u'tweet_urls', u'tweet_phone_numbers', u'tweet_dates_ref',
#        u'tweet_legislator_names', u'tweet_legislator_handles', u'tweet','total_score']]
# print len(final)
# print final_data.columns

today = datetime.date.today()
yr = str(today.year)[-2:]
mnth = today.month
if len(str(mnth)) == 1:
    mnth = '0' + str(mnth)
else:
    mnth = str(mnth)
day = today.day
if len(str(day)) == 1:
    day = '0' + str(day)
else:
    day = str(day)
    
filtered_data.drop(['pos_score','actionability_score'],1,inplace=True,errors='ignore')

csv_link = '~/w210_imwithdata/data/bucket_stuff/rzst_filtered_' + str(yr) + '.' + str(mnth) + '.' + str(day) +'.csv'
filtered_data.to_csv(csv_link,index=False)

/home/brosskatz/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [287]:
filtered_data['tweet'].tolist()[1500]

"TODAY ON THE ALEX JONES SHOW\n\nTuesday, March 28: Sanctuary Cities Battle Jeff Sessions - On today's broadcast we... https://t.co/1vpJWQdRr3"

In [228]:
print 'Total Score'
print np.mean(score_list)
print np.median(score_list)
print np.percentile(score_list,20)
print np.percentile(score_list,80)
print 
print 'ES Score'
print np.mean(es_score_list)
print np.median(es_score_list)
print np.percentile(es_score_list,20)
print np.percentile(es_score_list,90)
print 
print 'Action Score'
print np.mean(action_score_list)
print np.median(action_score_list)
print np.percentile(action_score_list,20)
print np.percentile(action_score_list,80)
print 
print 'Language Score'
print np.mean(language_score_list)
print np.median(language_score_list)
print np.percentile(language_score_list,20)
print np.percentile(language_score_list,80)
print 


Total Score
-1.47328537374
2.677262
-7.0983978
7.9324164

ES Score
4.6961256161
4.7697415
3.5385728
6.7605066

Action Score
-2.1923692134
1.0
-10.0
1.0

Language Score
-3.97704177644
1.0
-8.0
2.0

